In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import tensorflow as tf
import healpy as hp

from deepsphere import HealpyGCNN

from msfm.models.base_model import BaseModel

In [3]:
tf.print("test")

Metal device set to: Apple M1 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB

test


2023-02-14 10:49:28.509629: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-14 10:49:28.509765: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


### constants

In [4]:
dir_checkpoint = "./base_checkpoint"

# checkpointing tests

### setup a base model to be saved

In [5]:
network = tf.keras.Sequential([
    tf.keras.layers.InputLayer((None, 4)),
    tf.keras.layers.Dense(4)
    ])

model = BaseModel(network, (None, 2), dir_checkpoint=dir_checkpoint)
model.train_step.assign(10)
X = tf.random.uniform((10, 4), seed=1)
y = tf.random.uniform((10, 4), seed=2)
for _ in range(10):
    model.base_train_step(X, tf.keras.losses.mean_squared_error, y)

print(model.network.weights, "\n\n\n")
print(model.train_step, "\n\n\n")
print(model.optimizer.variables(), "\n\n\n")
model.save_model()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, None, 4)           20        
                                                                 
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________
23-02-14 10:49:28 base_model.p WAR   Performing a base_train_step in python instead of a tf.function 
23-02-14 10:49:28 base_model.p WAR   Performing a base_train_step in python instead of a tf.function 
23-02-14 10:49:28 base_model.p WAR   Performing a base_train_step in python instead of a tf.function 
23-02-14 10:49:28 base_model.p WAR   Performing a base_train_step in python instead of a tf.function 
23-02-14 10:49:28 base_model.p WAR   Performing a base_train_step in python instead of a tf.function 
23-02-14 10:49:28 base_model.p WAR   Performing a base_train_step in python in

### load that model
the variables should be identical

In [6]:
network = tf.keras.Sequential([
    tf.keras.layers.InputLayer((None, 4)),
    tf.keras.layers.Dense(4)
    ])

model_1 = BaseModel(network, (None, 2), dir_checkpoint=dir_checkpoint, restore_from_checkpoint=True)
print(model_1.network.weights, "\n\n\n")
print(model_1.train_step, "\n\n\n")
print(model_1.optimizer.variables(), "\n\n\n")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, None, 4)           20        
                                                                 
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________
23-02-14 10:49:29 base_model.p INF   Network successfully restored from checkpoint ./base_checkpoint/ckpt-1. 
[<tf.Variable 'dense_1/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[ 0.21377103,  0.47058317, -0.2539105 ,  0.52947545],
       [ 0.25066075, -0.28327873,  0.31893608, -0.5901518 ],
       [ 0.42085022,  0.4011252 , -0.12141883,  0.42287856],
       [-0.8038719 ,  0.7253791 , -0.5630019 ,  0.32778645]],
      dtype=float32)>, <tf.Variable 'dense_1/bias:0' shape=(4,) dtype=float32, numpy=array([ 0.00099983, -0.00099956,  0.00099987,  0.00099966], dtype=float32)>] 


### initialize from scratch
the variables should be different

In [7]:
network = tf.keras.Sequential([
    tf.keras.layers.InputLayer((None, 4)),
    tf.keras.layers.Dense(4)
    ])

model_2 = BaseModel(network, (None, 2), dir_checkpoint=dir_checkpoint, restore_from_checkpoint=False)
print(model_2.network.weights, "\n\n\n")
print(model_2.train_step, "\n\n\n")
print(model_2.optimizer.variables(), "\n\n\n")

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, None, 4)           20        
                                                                 
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________
23-02-14 10:49:29 base_model.p WAR   The model can not be saved when it is initialized from scratch with a non-empty checkpoint directory 
[<tf.Variable 'dense_2/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[-0.7698748 ,  0.48782164,  0.10950154, -0.2850235 ],
       [-0.01792341, -0.3727775 , -0.83747566,  0.6467523 ],
       [-0.44493857,  0.40461046, -0.29707783,  0.2723872 ],
       [-0.67028874, -0.05776989, -0.62274957,  0.07873529]],
      dtype=float32)>, <tf.Variable 'dense_2/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>] 



<tf.

### conflict handling

Trying to save the model creates another checkpoint, since it's a continuation of the original model

In [8]:
model_1.save_model()

23-02-14 10:49:29 base_model.p INF   Successfully saved the model to ./base_checkpoint 


Trying to save the model results in an error, since it was initialized from scratch

In [9]:
model_2.save_model()

Exception: The specified checkpoint directory ./base_checkpoint is not empty, can not save a model initialized from scratch there.